In [1]:
import os
import tensorflow as tf
import numpy as np
import gc
import pandas as pd
from datetime import datetime
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from tensorflow import keras

In [2]:
df = pd.read_csv("../Dataset/22-02-2018.csv", low_memory = False)

In [3]:
df = df.drop([0,1])

In [4]:
input_label = np.array(df.loc[:, df.columns != "Label"]).astype(np.float)

In [5]:
output_label = np.array(df["Label"])

In [6]:
out = []
for o in output_label:
    if(o == "Benign"):out.append(0)
    else: out.append(1)
output_label = out

In [7]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(input_label)
input_label = scaler.transform(input_label)

In [8]:
input_label, output_label = shuffle(input_label, output_label)

<h2>AutoEncoder</h2>

In [9]:
inp_train,inp_test,out_train,out_test = train_test_split(input_label, input_label, test_size=0.2)

In [10]:
input_model = keras.layers.Input(shape = (78,))
enc = keras.layers.Dense(units = 64, activation = "relu", use_bias = True)(input_model)
enc = keras.layers.Dense(units = 36, activation = "relu", use_bias = True)(enc)
enc = keras.layers.Dense(units = 18, activation = "relu")(enc)
dec = keras.layers.Dense(units = 36, activation = "relu", use_bias = True)(enc)
dec = keras.layers.Dense(units = 64, activation = "relu", use_bias = True)(dec)
dec = keras.layers.Dense(units = 78, activation = "relu", use_bias = True)(dec)
auto_encoder = keras.Model(input_model, dec)

In [11]:
encoder = keras.Model(input_model, enc)
decoder_input = keras.layers.Input(shape = (18,))
decoder_layer = auto_encoder.layers[-3](decoder_input)
decoder_layer = auto_encoder.layers[-2](decoder_layer)
decoder_layer = auto_encoder.layers[-1](decoder_layer)
decoder = keras.Model(decoder_input, decoder_layer)

In [12]:
auto_encoder.compile(optimizer=keras.optimizers.Adam(lr=0.00025), loss = "mean_squared_error", metrics = ['accuracy'])

In [13]:
train = auto_encoder.fit(x = np.array(inp_train), y = np.array(out_train),validation_split= 0.1, epochs = 10, verbose = 2, shuffle = True)

Epoch 1/10
23502/23502 [==============================] - 26s 1ms/step - loss: 0.0106 - accuracy: 0.7851 - val_loss: 0.0090 - val_accuracy: 0.6917
Epoch 2/10
23502/23502 [==============================] - 27s 1ms/step - loss: 0.0091 - accuracy: 0.8102 - val_loss: 0.0090 - val_accuracy: 0.6680
Epoch 3/10
23502/23502 [==============================] - 25s 1ms/step - loss: 0.0091 - accuracy: 0.8121 - val_loss: 0.0090 - val_accuracy: 0.9057
Epoch 4/10
23502/23502 [==============================] - 24s 1ms/step - loss: 0.0091 - accuracy: 0.8124 - val_loss: 0.0090 - val_accuracy: 0.8691
Epoch 5/10
23502/23502 [==============================] - 27s 1ms/step - loss: 0.0091 - accuracy: 0.8103 - val_loss: 0.0090 - val_accuracy: 0.8360
Epoch 6/10
23502/23502 [==============================] - 24s 1ms/step - loss: 0.0091 - accuracy: 0.8118 - val_loss: 0.0090 - val_accuracy: 0.6939
Epoch 7/10
23502/23502 [==============================] - 27s 1ms/step - loss: 0.0090 - accuracy: 0.8128 - val_loss: 0

<h2>cross validation</h2>

In [17]:
input_label = encoder.predict(input_label).reshape(len(input_label), 18, 1)

In [18]:
def createModel():
    model = keras.Sequential([
        keras.layers.Conv1D(filters = 16, input_shape = (18,1), kernel_size = 3, padding = "same", activation = "relu", use_bias = True),
        keras.layers.MaxPool1D(pool_size = 3),
        keras.layers.Conv1D(filters = 8, kernel_size = 3, padding = "same", activation = "relu", use_bias = True),
        keras.layers.MaxPool1D(pool_size = 3),
        keras.layers.Flatten(),
        keras.layers.Dense(units = 2, activation = "softmax")
    ])
    model.compile(optimizer= keras.optimizers.Adam(lr= 0.00025), loss="sparse_categorical_crossentropy", metrics=['accuracy'])
    return model

In [19]:
skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state=1)

In [20]:
confusion_matrixs = []
roc_curvs = []

In [21]:
for i, (train, test) in enumerate(skf.split(input_label, output_label)):
    print("Modelo " + str(i))
    inp_train, out_train = np.array(input_label)[train], np.array(output_label)[train]
    inp_test, out_test = np.array(input_label)[test], np.array(output_label)[test]
    model = createModel()
    model.fit(x = inp_train, y = out_train, validation_split= 0.1, epochs = 10, shuffle = True,verbose = 2)
    res = np.array([np.argmax(resu) for resu in model.predict(inp_test)])
    confusion_matrixs.append(confusion_matrix(out_test, res))
    fpr, tpr, _ = roc_curve(out_test,  res)
    auc = roc_auc_score(out_test, res)
    roc_curvs.append([fpr, tpr, auc])
    print("\n\n")

Modelo0
Epoch 1/10
26440/26440 [==============================] - 37s 1ms/step - loss: 0.1398 - accuracy: 0.9412 - val_loss: 0.0084 - val_accuracy: 0.9978
Epoch 2/10
26440/26440 [==============================] - 36s 1ms/step - loss: 0.0077 - accuracy: 0.9979 - val_loss: 0.0064 - val_accuracy: 0.9991
Epoch 3/10
26440/26440 [==============================] - 36s 1ms/step - loss: 0.0046 - accuracy: 0.9986 - val_loss: 0.0034 - val_accuracy: 0.9989
Epoch 4/10
26440/26440 [==============================] - 36s 1ms/step - loss: 0.0035 - accuracy: 0.9990 - val_loss: 0.0024 - val_accuracy: 0.9994
Epoch 5/10
26440/26440 [==============================] - 36s 1ms/step - loss: 0.0029 - accuracy: 0.9993 - val_loss: 0.0022 - val_accuracy: 0.9994
Epoch 6/10
26440/26440 [==============================] - 36s 1ms/step - loss: 0.0023 - accuracy: 0.9994 - val_loss: 0.0017 - val_accuracy: 0.9996
Epoch 7/10
26440/26440 [==============================] - 36s 1ms/step - loss: 0.0020 - accuracy: 0.9995 - val

KeyboardInterrupt: 

<h2>Roc Curves</h2>

In [ ]:
for i in range(10):
    print("------------------------------------")
    print("Modelo " + str(i))
    print(roc_curvs[i])
    print(confusion_matrixs[i])
    print("------------------------------------")